Code pour récupérer toutes mes données Strava + Préparation de base de données pour analyser mes activités de "Running" pour un DashBoard (Looker)

Code en cours de modification.

Looker Studio : https://lookerstudio.google.com/reporting/c6a61b3a-7a37-423d-a725-44d4daf62fe4

In [63]:
import requests

client_id = 168568
client_secret = "b1a8236527721977c9b839e02468f5d64428d5bd"
code = "6a782f72f35ada6daf6d6bf2907756c77e56e44e"

response = requests.post(
    "https://www.strava.com/oauth/token",
    data={
        "client_id": client_id,
        "client_secret": client_secret,
        "code": code,
        "grant_type": "authorization_code"
    }
)

token_data = response.json()

print("✅ Access Token :", token_data.get("access_token"))
print("🔁 Refresh Token :", token_data.get("refresh_token"))
print("📅 Expiration timestamp :", token_data.get("expires_at"))


✅ Access Token : e65d0c90540d25c7177cc630ce811cbe00727a09
🔁 Refresh Token : a880ebed7100faff4e4a2447bd5d30e67c001fcb
📅 Expiration timestamp : 1752773063


In [64]:
import time
import pandas as pd

# Récupération du token obtenu juste avant
access_token = token_data.get("access_token")

headers = {
    "Authorization": f"Bearer {access_token}"
}

all_activities = []
page = 1

print("\n📦 Téléchargement des activités Strava en cours...")

while True:
    response = requests.get(
        "https://www.strava.com/api/v3/athlete/activities",
        headers=headers,
        params={
            "per_page": 200,
            "page": page
        }
    )

    if response.status_code != 200:
        print(f"❌ Erreur {response.status_code} : {response.text}")
        break

    activities = response.json()

    if not activities:
        print("✅ Toutes les activités ont été récupérées.")
        break

    all_activities.extend(activities)
    print(f"📄 Page {page} : {len(activities)} activités ajoutées.")
    page += 1
    time.sleep(1)

# Transformation en DataFrame
df = pd.DataFrame(all_activities)



📦 Téléchargement des activités Strava en cours...
📄 Page 1 : 200 activités ajoutées.
📄 Page 2 : 200 activités ajoutées.
📄 Page 3 : 200 activités ajoutées.
📄 Page 4 : 176 activités ajoutées.
✅ Toutes les activités ont été récupérées.


In [65]:
df_run = df[df['type'] == 'Run'].copy()





In [66]:
df_run['sortie_longue'] = df_run["name"].str.lower().str.contains("sl|sortie longue")

In [67]:
df_run["date"] = pd.to_datetime(df["start_date_local"].str[:10])


In [68]:
df_run["distance"] = df_run["distance"] / 1000

df_run['average_speed'] = df_run['average_speed'] * 3.6

In [69]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# Authentification
creds, _ = default()
gc = gspread.authorize(creds)

# 🔍 Vérifie si le fichier existe déjà
spreadsheet_name = "Strava_Run_Exportv2"
try:
    spreadsheet = gc.open(spreadsheet_name)
    print(f"📄 Fichier '{spreadsheet_name}' trouvé. Mise à jour en cours.")
except gspread.exceptions.SpreadsheetNotFound:
    spreadsheet = gc.create(spreadsheet_name)
    print(f"📄 Fichier '{spreadsheet_name}' créé.")

# 📌 Récupère ou crée l'onglet
try:
    worksheet = spreadsheet.worksheet("Sheet1")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title="Sheet1", rows=1000, cols=20)

# 🧹 Vide l'onglet existant
worksheet.clear()

# ⬆️ Envoie les données
set_with_dataframe(worksheet, df_run)

print(f"✅ 'df_run' exporté avec succès dans le fichier Google Sheets : '{spreadsheet_name}'")



📄 Fichier 'Strava_Run_Exportv2' trouvé. Mise à jour en cours.
✅ 'df_run' exporté avec succès dans le fichier Google Sheets : 'Strava_Run_Exportv2'
